In [1]:
import psycopg
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import json
from tqdm.notebook import tqdm
import seaborn as sns


# --- LOOKUP FILES ---
LOOKUPS = json.load(open('./json/lookups.json', 'r'))

# --- CONFIGURATION ---
POSTFIX = '03_25'
COURT = 'HSG'
START_DATE = '2024-09-11'

# --- TABLE NAMES (auto-generated based on POSTFIX) ---
schedule_table = f"foia_schedule_{POSTFIX}"
case_table = f"foia_case_{POSTFIX}"
proceeding_table = f"foia_proceeding_{POSTFIX}"
rider_table = f"foia_rider_{POSTFIX}"
charge_table = f"foia_charges_{POSTFIX}"


# Database connection parameters - you'll need to fill these in
db_params = {
    'dbname': 'bklg_test',  # Based on your schema info
    'user': 'bklg',             # Fill in your username
    'password': 'devpassword',         # Fill in your password
    'host': 'localhost',    # Change if needed
    'port': '5432'          # Change if needed
}

# Function to connect to the database
def connect_to_db(params):
    """Establish connection to the PostgreSQL database"""
    try:
        conn = psycopg.connect(**params)
        print("Database connection established successfully")
        return conn
    except Exception as e:
        print(f"Error connecting to the database: {e}")
        return None


conn = connect_to_db(db_params)
conn


Database connection established successfully


<psycopg.Connection [IDLE] (host=localhost user=bklg database=bklg_test) at 0x7b25e8167eb0>

In [2]:
# Query to get maximum input_date for cal_type 'I' at the specified court
query_get_end_date = f"""
SELECT MAX(input_date)::date AS max_input_date
FROM {schedule_table}
WHERE base_city_code = %(court)s
  AND cal_type = 'I'
"""

with connect_to_db(db_params) as conn:
    result = pd.read_sql(query_get_end_date, conn, params={'court': COURT})
    DUMPED_DATE = result.loc[0, 'max_input_date']
    print(f"Using Dumped Date: {DUMPED_DATE}")

Database connection established successfully
Using Dumped Date: 2025-02-28


/tmp/ipykernel_76054/2005720099.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result = pd.read_sql(query_get_end_date, conn, params={'court': COURT})


In [3]:
# Connect to the database
conn = connect_to_db(db_params)

# Load the materialized view directly into a DataFrame
caseloads_df = pd.read_sql("SELECT * FROM judge_caseloads_at_scheduling_since_02_2022", conn)

# Optional: Convert datetime column
caseloads_df['scheduled_datetime'] = pd.to_datetime(caseloads_df['scheduled_datetime'])

# Display basic info
print(f"Loaded {len(caseloads_df)} rows with shape {caseloads_df.shape}")

Database connection established successfully


/tmp/ipykernel_76054/3116039534.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  caseloads_df = pd.read_sql("SELECT * FROM judge_caseloads_at_scheduling_since_02_2022", conn)


Loaded 2358660 rows with shape (2358660, 11)


In [4]:
percent_missing = caseloads_df.isnull().sum() * 100 / len(caseloads_df)
percent_missing

idnschedule           0.000000
idncase               0.000000
idnproceeding         0.000000
assigned_judge        0.000000
scheduled_date        0.000000
scheduled_time        0.005088
scheduled_datetime    0.005088
hearing_date          0.000000
hearing_time          0.000000
judge                 0.000000
caseload              0.000000
dtype: float64

In [5]:
cdf = caseloads_df
mask = cdf['scheduled_datetime'].isna()
cdf.loc[mask, 'scheduled_datetime'] = pd.to_datetime(cdf.loc[mask, 'scheduled_date']) + pd.Timedelta(hours=0)
cdf[cdf['scheduled_time'].isna()]

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload
149280,41366453,10256456,10375254,CMS,2022-04-28,None,2022-04-28,2022-09-06,10:30:00,AAJ,724
149281,41366453,10256456,10375254,CMS,2022-04-28,None,2022-04-28,2022-09-06,10:30:00,AKN,0
149282,41366453,10256456,10375254,CMS,2022-04-28,None,2022-04-28,2022-09-06,10:30:00,AWS,0
149283,41366453,10256456,10375254,CMS,2022-04-28,None,2022-04-28,2022-09-06,10:30:00,BLH,0
149284,41366453,10256456,10375254,CMS,2022-04-28,None,2022-04-28,2022-09-06,10:30:00,CBA,0
...,...,...,...,...,...,...,...,...,...,...,...
332575,42460774,10653599,11146987,CMR,2022-08-18,None,2022-08-18,2023-11-03,10:30:00,VJ6,0
332576,42460774,10653599,11146987,CMR,2022-08-18,None,2022-08-18,2023-11-03,10:30:00,VJ7,4
332577,42460774,10653599,11146987,CMR,2022-08-18,None,2022-08-18,2023-11-03,10:30:00,VJ8,0
332578,42460774,10653599,11146987,CMR,2022-08-18,None,2022-08-18,2023-11-03,10:30:00,WCP,0


In [6]:
g = caseloads_df.groupby('idncase')
g.size().value_counts()

60     17287
120     5995
180     2102
240      607
300      188
360       48
420        9
540        1
Name: count, dtype: int64

In [7]:
cdf.head(20)

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,832
1,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AKN,0
2,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AWS,0
3,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,BLH,0
4,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CBA,0
5,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CDK,775
6,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CLR,0
7,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CMR,831
8,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CMS,875
9,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CMW,0


In [8]:
cdf[(cdf['assigned_judge'] == cdf['judge']) & (cdf['judge'] == 'AWS')].head(50)

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload
910022,45160248,10355109,10488609,AWS,2023-05-22,09:55:00,2023-05-22 09:55:00,2024-11-18,08:30:00,AWS,1
910502,45160461,10482641,10635129,AWS,2023-05-22,10:08:00,2023-05-22 10:08:00,2024-11-19,08:30:00,AWS,2
932222,45237228,11138090,11424168,AWS,2023-05-29,17:43:00,2023-05-29 17:43:00,2024-11-21,13:30:00,AWS,3
932342,45237326,11656925,12044308,AWS,2023-05-29,18:10:00,2023-05-29 18:10:00,2024-11-21,08:30:00,AWS,5
932402,45237327,11656935,12044321,AWS,2023-05-29,18:10:00,2023-05-29 18:10:00,2024-11-21,08:30:00,AWS,5
935702,45242464,11578721,12317192,AWS,2023-05-30,11:29:00,2023-05-30 11:29:00,2024-11-14,13:30:00,AWS,6
936002,45244498,10581479,11367966,AWS,2023-05-30,12:49:00,2023-05-30 12:49:00,2024-11-14,08:30:00,AWS,7
936902,45248293,8993149,12317850,AWS,2023-05-30,15:54:00,2023-05-30 15:54:00,2024-11-12,08:30:00,AWS,8
943262,45263215,10865562,11096175,AWS,2023-05-31,16:01:00,2023-05-31 16:01:00,2024-12-02,13:30:00,AWS,11
943322,45263216,10865567,11096180,AWS,2023-05-31,16:01:00,2023-05-31 16:01:00,2024-12-02,13:30:00,AWS,11


In [9]:
# Get the minimum idnschedule for each unique combination of scheduled_datetime and judge
family_size_df = cdf.groupby(['scheduled_datetime', 'judge']).size().reset_index(name='family_size')
family_size_df.head(70)

cdf = cdf.merge(family_size_df, on=['scheduled_datetime', 'judge'])

min_idnschedule_df = cdf.groupby(['scheduled_datetime', 'judge'])['idnschedule'].min().reset_index()

# Merge with the original DataFrame to keep only the records with the minimum idnschedule
cdf = cdf.merge(min_idnschedule_df, on=['scheduled_datetime', 'judge', 'idnschedule'])
cdf.sort_values('scheduled_datetime').reset_index(drop=True)
cdf[cdf['judge'] == cdf['assigned_judge']]

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload,family_size
37,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,MGC,765,1
97,40518481,9688070,9664462,MGC,2022-02-01,09:19:00,2022-02-01 09:19:00,2023-03-03,08:30:00,MGC,767,2
153,40519739,9526295,9826177,KBR,2022-02-01,10:33:00,2022-02-01 10:33:00,2023-03-08,08:30:00,KBR,762,2
213,40520010,9449167,9352370,KBR,2022-02-01,10:42:00,2022-02-01 10:42:00,2023-03-08,10:00:00,KBR,764,2
273,40520030,9319288,9382541,KBR,2022-02-01,10:44:00,2022-02-01 10:44:00,2023-03-08,13:00:00,KBR,765,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1509757,53588018,14187610,14933900,MGC,2025-02-28,14:05:00,2025-02-28 14:05:00,2025-07-24,08:30:00,MGC,209,1
1509790,53588520,14129041,14863464,DAJ,2025-02-28,14:23:00,2025-02-28 14:23:00,2025-09-04,08:30:00,DAJ,1461,5
1509850,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,DAJ,1461,1
1509937,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,MGC,210,1


In [10]:
cdf['family_size'].value_counts()

family_size
1     1082400
2      211380
3      126180
4       57120
5       20460
6        4200
7        1260
8        1020
9         720
20        540
12        480
18        480
14        480
10        420
11        300
17        300
19        300
15        240
26        240
25        180
23        180
22        180
24        180
13        120
16        120
21        120
28        120
27         60
31         60
34         60
29         60
32         60
Name: count, dtype: int64

In [11]:

# 1. Ensure scheduled_datetime is a datetime object (if not already)
cdf['scheduled_datetime'] = pd.to_datetime(cdf['scheduled_datetime'])

# 2. Combine hearing_date and hearing_time into hearing_datetime
cdf['hearing_datetime'] = pd.to_datetime(cdf['hearing_date'].astype(str) + ' ' + cdf['hearing_time'].astype(str))

# 3. Create an auxiliary column that is only populated when the judge is assigned
cdf['assigned_hearing'] = cdf.apply(lambda row: row['hearing_datetime'] if row['judge'] == row['assigned_judge'] else pd.NaT, axis=1)
cdf.head(40)

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload,family_size,hearing_datetime,assigned_hearing
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,832,1,2023-02-13 10:30:00,NaT
1,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AKN,0,1,2023-02-13 10:30:00,NaT
2,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AWS,0,1,2023-02-13 10:30:00,NaT
3,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,BLH,0,1,2023-02-13 10:30:00,NaT
4,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CBA,0,1,2023-02-13 10:30:00,NaT
5,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CDK,775,1,2023-02-13 10:30:00,NaT
6,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CLR,0,1,2023-02-13 10:30:00,NaT
7,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CMR,831,1,2023-02-13 10:30:00,NaT
8,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CMS,875,1,2023-02-13 10:30:00,NaT
9,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CMW,0,1,2023-02-13 10:30:00,NaT


In [12]:
# 4. Sort by judge and scheduled_datetime so that we can look ahead correctly within each judge's records
cdf = cdf.sort_values(by=['judge', 'scheduled_datetime'])
cdf.head(30)

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload,family_size,hearing_datetime,assigned_hearing
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,832,1,2023-02-13 10:30:00,NaT
60,40518481,9688070,9664462,MGC,2022-02-01,09:19:00,2022-02-01 09:19:00,2023-03-03,08:30:00,AAJ,832,2,2023-03-03 08:30:00,NaT
120,40519739,9526295,9826177,KBR,2022-02-01,10:33:00,2022-02-01 10:33:00,2023-03-08,08:30:00,AAJ,832,2,2023-03-08 08:30:00,NaT
180,40520010,9449167,9352370,KBR,2022-02-01,10:42:00,2022-02-01 10:42:00,2023-03-08,10:00:00,AAJ,832,2,2023-03-08 10:00:00,NaT
240,40520030,9319288,9382541,KBR,2022-02-01,10:44:00,2022-02-01 10:44:00,2023-03-08,13:00:00,AAJ,832,1,2023-03-08 13:00:00,NaT
300,40520058,9772005,9814430,KBR,2022-02-01,10:46:00,2022-02-01 10:46:00,2023-03-13,08:30:00,AAJ,832,1,2023-03-13 08:30:00,NaT
360,40520082,9672363,9642627,KBR,2022-02-01,10:47:00,2022-02-01 10:47:00,2023-03-13,10:00:00,AAJ,832,1,2023-03-13 10:00:00,NaT
420,40520109,9203836,9051596,KBR,2022-02-01,10:49:00,2022-02-01 10:49:00,2023-03-13,13:00:00,AAJ,832,1,2023-03-13 13:00:00,NaT
480,40520191,9466552,9404140,KBR,2022-02-01,10:52:00,2022-02-01 10:52:00,2023-03-14,10:00:00,AAJ,832,1,2023-03-14 10:00:00,NaT
540,40520264,9064265,9358421,KBR,2022-02-01,10:53:00,2022-02-01 10:53:00,2023-03-14,13:00:00,AAJ,832,1,2023-03-14 13:00:00,NaT


In [13]:
# 5. For each judge, fill NaT values in 'assigned_hearing' with the next available assigned hearing time.
#    This gives each record the next hearing time where the judge is actually assigned.
cdf['next_assigned_hearing'] = cdf.groupby('judge')['assigned_hearing'].fillna(method='bfill')
cdf

/tmp/ipykernel_76054/1828205327.py:3: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  cdf['next_assigned_hearing'] = cdf.groupby('judge')['assigned_hearing'].fillna(method='bfill')
/tmp/ipykernel_76054/1828205327.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  cdf['next_assigned_hearing'] = cdf.groupby('judge')['assigned_hearing'].fillna(method='bfill')


,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload,family_size,hearing_datetime,assigned_hearing,next_assigned_hearing
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,832,1,2023-02-13 10:30:00,NaT,2022-12-12 13:00:00
60,40518481,9688070,9664462,MGC,2022-02-01,09:19:00,2022-02-01 09:19:00,2023-03-03,08:30:00,AAJ,832,2,2023-03-03 08:30:00,NaT,2022-12-12 13:00:00
120,40519739,9526295,9826177,KBR,2022-02-01,10:33:00,2022-02-01 10:33:00,2023-03-08,08:30:00,AAJ,832,2,2023-03-08 08:30:00,NaT,2022-12-12 13:00:00
180,40520010,9449167,9352370,KBR,2022-02-01,10:42:00,2022-02-01 10:42:00,2023-03-08,10:00:00,AAJ,832,2,2023-03-08 10:00:00,NaT,2022-12-12 13:00:00
240,40520030,9319288,9382541,KBR,2022-02-01,10:44:00,2022-02-01 10:44:00,2023-03-08,13:00:00,AAJ,832,1,2023-03-08 13:00:00,NaT,2022-12-12 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509779,53588018,14187610,14933900,MGC,2025-02-28,14:05:00,2025-02-28 14:05:00,2025-07-24,08:30:00,WTN,0,1,2025-07-24 08:30:00,NaT,NaT
1509839,53588520,14129041,14863464,DAJ,2025-02-28,14:23:00,2025-02-28 14:23:00,2025-09-04,08:30:00,WTN,0,5,2025-09-04 08:30:00,NaT,NaT
1509899,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,WTN,0,1,2025-05-01 10:30:00,NaT,NaT
1509959,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,WTN,0,1,2025-11-20 13:00:00,NaT,NaT


In [14]:
# 6. Compute the time difference in Unix seconds between the next assigned hearing and the current scheduled time.
#    For the row where the judge is assigned, this is the current hearing time; for others, it is the next one.
cdf['time_to_next'] = (cdf['next_assigned_hearing'] - cdf['scheduled_datetime']).dt.total_seconds()

#cdf = cdf.groupby('judge').filter(lambda group: group['caseload'].max() > 0)

# 7. (Optional) Sort back by scheduled_datetime if needed.
cdf = cdf.sort_values(by='scheduled_datetime').reset_index(drop=True)
cdf.head(20)

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload,family_size,hearing_datetime,assigned_hearing,next_assigned_hearing,time_to_next
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,832,1,2023-02-13 10:30:00,NaT,2022-12-12 13:00:00,27143340.0
1,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,RB,0,1,2023-02-13 10:30:00,NaT,NaT,NaN
2,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,DMP,0,1,2023-02-13 10:30:00,NaT,2023-07-20 13:00:00,46151340.0
3,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,JBE,0,1,2023-02-13 10:30:00,NaT,NaT,NaN
4,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CLR,0,1,2023-02-13 10:30:00,NaT,NaT,NaN
5,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AWS,0,1,2023-02-13 10:30:00,NaT,2024-11-18 08:30:00,88211940.0
6,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,VJ8,1,1,2023-02-13 10:30:00,NaT,NaT,NaN
7,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,GYH,112,1,2023-02-13 10:30:00,NaT,2022-12-06 13:30:00,26626740.0
8,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,VJ6,3,1,2023-02-13 10:30:00,NaT,NaT,NaN
9,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,JLP,0,1,2023-02-13 10:30:00,NaT,NaT,NaN


In [15]:
cdf.shape

(1510020, 16)

In [16]:
g = cdf.groupby('idncase')
g.size().value_counts()

60     12877
120     3841
180     1072
240      230
300       65
360       21
420        3
Name: count, dtype: int64

In [17]:
# Find records where AAJ is the assigned judge 
aaj_assigned = cdf[(cdf['judge'] == 'AAJ') & (cdf['assigned_judge'] == 'AAJ')].sort_values('scheduled_datetime')
print("Records where AAJ is the assigned judge:")
aaj_assigned[['idnschedule', 'scheduled_datetime', 'hearing_datetime', 'assigned_hearing', 'next_assigned_hearing', 'time_to_next']].head(20)


Records where AAJ is the assigned judge:


,idnschedule,scheduled_datetime,hearing_datetime,assigned_hearing,next_assigned_hearing,time_to_next
4674,40535515,2022-02-02 14:18:00,2022-12-12 13:00:00,2022-12-12 13:00:00,2022-12-12 13:00:00,27038520.0
4681,40535601,2022-02-02 14:24:00,2022-08-16 08:30:00,2022-08-16 08:30:00,2022-08-16 08:30:00,16826760.0
4811,40535689,2022-02-02 14:30:00,2022-12-09 10:00:00,2022-12-09 10:00:00,2022-12-09 10:00:00,26767800.0
4974,40536101,2022-02-02 14:54:00,2023-03-10 13:00:00,2023-03-10 13:00:00,2023-03-10 13:00:00,34639560.0
4994,40536356,2022-02-02 15:09:00,2023-03-02 10:00:00,2023-03-02 10:00:00,2023-03-02 10:00:00,33936660.0
6602,40550807,2022-02-03 17:21:00,2023-03-14 10:00:00,2023-03-14 10:00:00,2023-03-14 10:00:00,34879140.0
6681,40550829,2022-02-03 17:24:00,2022-04-26 13:00:00,2022-04-26 13:00:00,2022-04-26 13:00:00,7068960.0
6737,40550886,2022-02-03 17:30:00,2023-01-26 13:00:00,2023-01-26 13:00:00,2023-01-26 13:00:00,30828600.0
9089,40565100,2022-02-07 09:58:00,2023-03-14 13:00:00,2023-03-14 13:00:00,2023-03-14 13:00:00,34570920.0
10600,40590263,2022-02-09 11:22:00,2022-02-18 08:30:00,2022-02-18 08:30:00,2022-02-18 08:30:00,767280.0


In [18]:
cdf[cdf['judge'] == 'AAJ'].sort_values('scheduled_datetime').reset_index(drop=True)

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload,family_size,hearing_datetime,assigned_hearing,next_assigned_hearing,time_to_next
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,832,1,2023-02-13 10:30:00,NaT,2022-12-12 13:00:00,27143340.0
1,40518481,9688070,9664462,MGC,2022-02-01,09:19:00,2022-02-01 09:19:00,2023-03-03,08:30:00,AAJ,832,2,2023-03-03 08:30:00,NaT,2022-12-12 13:00:00,27142860.0
2,40519739,9526295,9826177,KBR,2022-02-01,10:33:00,2022-02-01 10:33:00,2023-03-08,08:30:00,AAJ,832,2,2023-03-08 08:30:00,NaT,2022-12-12 13:00:00,27138420.0
3,40520010,9449167,9352370,KBR,2022-02-01,10:42:00,2022-02-01 10:42:00,2023-03-08,10:00:00,AAJ,832,2,2023-03-08 10:00:00,NaT,2022-12-12 13:00:00,27137880.0
4,40520030,9319288,9382541,KBR,2022-02-01,10:44:00,2022-02-01 10:44:00,2023-03-08,13:00:00,AAJ,832,1,2023-03-08 13:00:00,NaT,2022-12-12 13:00:00,27137760.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25162,53588018,14187610,14933900,MGC,2025-02-28,14:05:00,2025-02-28 14:05:00,2025-07-24,08:30:00,AAJ,985,1,2025-07-24 08:30:00,NaT,NaT,NaN
25163,53588520,14129041,14863464,DAJ,2025-02-28,14:23:00,2025-02-28 14:23:00,2025-09-04,08:30:00,AAJ,985,5,2025-09-04 08:30:00,NaT,NaT,NaN
25164,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,AAJ,985,1,2025-05-01 10:30:00,NaT,NaT,NaN
25165,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,AAJ,985,1,2025-11-20 13:00:00,NaT,NaT,NaN


In [19]:
cdf['assigned'] = cdf.apply(lambda row: 1 if row['assigned_judge'] == row['judge'] else 0, axis=1)
cdf

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload,family_size,hearing_datetime,assigned_hearing,next_assigned_hearing,time_to_next,assigned
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,832,1,2023-02-13 10:30:00,NaT,2022-12-12 13:00:00,27143340.0,0
1,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,RB,0,1,2023-02-13 10:30:00,NaT,NaT,NaN,0
2,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,DMP,0,1,2023-02-13 10:30:00,NaT,2023-07-20 13:00:00,46151340.0,0
3,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,JBE,0,1,2023-02-13 10:30:00,NaT,NaT,NaN,0
4,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,CLR,0,1,2023-02-13 10:30:00,NaT,NaT,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510015,53591844,13604711,14226348,AWS,2025-02-28,17:06:00,2025-02-28 17:06:00,2027-07-20,10:30:00,KBR,1585,3,2027-07-20 10:30:00,NaT,NaT,NaN,0
1510016,53591844,13604711,14226348,AWS,2025-02-28,17:06:00,2025-02-28 17:06:00,2027-07-20,10:30:00,JNM,0,3,2027-07-20 10:30:00,NaT,NaT,NaN,0
1510017,53591844,13604711,14226348,AWS,2025-02-28,17:06:00,2025-02-28 17:06:00,2027-07-20,10:30:00,JNC,0,3,2027-07-20 10:30:00,NaT,NaT,NaN,0
1510018,53591844,13604711,14226348,AWS,2025-02-28,17:06:00,2025-02-28 17:06:00,2027-07-20,10:30:00,RB,0,3,2027-07-20 10:30:00,NaT,NaT,NaN,0


In [20]:
percent_missing = cdf.isnull().sum() * 100 / len(cdf)
percent_missing

idnschedule               0.000000
idncase                   0.000000
idnproceeding             0.000000
assigned_judge            0.000000
scheduled_date            0.000000
scheduled_time            0.007947
scheduled_datetime        0.000000
hearing_date              0.000000
hearing_time              0.000000
judge                     0.000000
caseload                  0.000000
family_size               0.000000
hearing_datetime          0.000000
assigned_hearing         98.333333
next_assigned_hearing    77.274672
time_to_next             77.274672
assigned                  0.000000
dtype: float64

In [21]:
cdf.groupby('judge')['time_to_next'].apply(lambda x: x.isnull().all())

judge
AAJ    False
AKN     True
AWS    False
BLH     True
CBA     True
CDK    False
CLR     True
CMR    False
CMS    False
CMW     True
DAJ    False
DDB     True
DER     True
DIG     True
DMP    False
DRN     True
ETK     True
FGL     True
GYH    False
HR      True
HRG     True
JAD     True
JBE     True
JBN     True
JBT     True
JDA    False
JDO     True
JGC    False
JJC    False
JLB     True
JLP     True
JNC     True
JNM    False
KBR    False
KC      True
KYT    False
LLU     True
MGC    False
MGM     True
MIO     True
MKS     True
MSY     True
NES    False
RB      True
RLR     True
RR1     True
RSH     True
RW      True
SEG     True
SG1     True
SME     True
TCR     True
VJ2     True
VJ4     True
VJ5     True
VJ6     True
VJ7     True
VJ8     True
WCP     True
WTN     True
Name: time_to_next, dtype: bool

In [22]:
cdf.groupby('judge')['caseload'].apply(lambda x: (x==0).all())

judge
AAJ    False
AKN     True
AWS    False
BLH     True
CBA     True
CDK    False
CLR     True
CMR    False
CMS    False
CMW     True
DAJ    False
DDB     True
DER     True
DIG     True
DMP    False
DRN     True
ETK     True
FGL     True
GYH    False
HR      True
HRG     True
JAD     True
JBE     True
JBN     True
JBT     True
JDA    False
JDO     True
JGC    False
JJC    False
JLB     True
JLP     True
JNC     True
JNM    False
KBR    False
KC      True
KYT    False
LLU     True
MGC    False
MGM     True
MIO     True
MKS     True
MSY     True
NES    False
RB      True
RLR     True
RR1     True
RSH     True
RW      True
SEG     True
SG1    False
SME     True
TCR     True
VJ2     True
VJ4     True
VJ5     True
VJ6    False
VJ7    False
VJ8    False
WCP     True
WTN     True
Name: caseload, dtype: bool

In [23]:
null_time_series = cdf.groupby('judge')['time_to_next'].apply(lambda x: x.isnull().all())
zero_caseload_series = cdf.groupby('judge')['caseload'].apply(lambda x: (x==0).all())

# Building the dataframe with both series
result_df = pd.DataFrame({
    'always_null_time': null_time_series,
    'always_zero_caseload': zero_caseload_series
})

# Adding the third series that's true if one is true and the other is false
result_df['one_true_one_false'] = (result_df['always_null_time'] ^ result_df['always_zero_caseload'])

# Display the result
print(result_df)

       always_null_time  always_zero_caseload  one_true_one_false
judge                                                            
AAJ               False                 False               False
AKN                True                  True               False
AWS               False                 False               False
BLH                True                  True               False
CBA                True                  True               False
CDK               False                 False               False
CLR                True                  True               False
CMR               False                 False               False
CMS               False                 False               False
CMW                True                  True               False
DAJ               False                 False               False
DDB                True                  True               False
DER                True                  True               False
DIG       

In [24]:
# Get the list of judges who have one true and one false
judges_one_true_one_false = result_df[result_df['one_true_one_false']].index.tolist()

# Print the list of such judges
print("Judges with exactly one true condition:")
print(judges_one_true_one_false)

# For each of these judges, inspect their original records in the dataframe
for judge in judges_one_true_one_false:
    print(f"\n--- Records for {judge} ---")
    judge_records = cdf[cdf['judge'] == judge]
    
    # Print basic stats for this judge
    print(f"Number of records: {len(judge_records)}")
    print(f"All time_to_next are null: {result_df.loc[judge, 'always_null_time']}")
    print(f"All caseload are zero: {result_df.loc[judge, 'always_zero_caseload']}")
    
    # Display a sample of the records for this judge
    print("\nSample records:")
    print(judge_records[['idncase', 'caseload', 'time_to_next']].head())
    
    # Show basic statistics for this judge's caseload and time_to_next
    print("\nBasic statistics for caseload:")
    print(judge_records['caseload'].describe())
    
    if not result_df.loc[judge, 'always_null_time']:
        print("\nBasic statistics for non-null time_to_next values:")
        print(judge_records['time_to_next'].dropna().describe())

Judges with exactly one true condition:
['SG1', 'VJ6', 'VJ7', 'VJ8']

--- Records for SG1 ---
Number of records: 25167
All time_to_next are null: True
All caseload are zero: False

Sample records:
     idncase  caseload  time_to_next
20   7356844        88           NaN
118  9688070        88           NaN
164  9526295        88           NaN
215  9449167        88           NaN
275  9319288        88           NaN

Basic statistics for caseload:
count    25167.000000
mean         2.776811
std         11.161504
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         88.000000
Name: caseload, dtype: float64

--- Records for VJ6 ---
Number of records: 25167
All time_to_next are null: True
All caseload are zero: False

Sample records:
     idncase  caseload  time_to_next
8    7356844         3           NaN
79   9688070         3           NaN
156  9526295         3           NaN
187  9449167         3           NaN
248  9319288         3       

In [25]:
# Get the list of judges who always have null time_to_next
judges_with_null_time = result_df[result_df['always_null_time']].index.tolist()

# Print the judges being removed
print(f"Removing records for {len(judges_with_null_time)} judges with always null time_to_next:")
print(judges_with_null_time)

# Create a new dataframe without these judges
filtered_df = cdf[~cdf['judge'].isin(judges_with_null_time)].copy()

# Print before and after record counts
print(f"\nOriginal dataframe: {len(cdf)} records")
print(f"Filtered dataframe: {len(filtered_df)} records")
print(f"Removed {len(cdf) - len(filtered_df)} records")

# Quick verification - check if any of the filtered judges remain
judges_remaining = filtered_df['judge'].unique()
print(f"\nNumber of unique judges in filtered dataframe: {len(judges_remaining)}")

# Verify none of the always_null_time judges remain
overlap = set(judges_with_null_time) & set(judges_remaining)
if overlap:
    print(f"Warning: {len(overlap)} judges with always_null_time still in filtered data: {overlap}")
else:
    print("Verification successful: All judges with always_null_time have been removed")

Removing records for 44 judges with always null time_to_next:
['AKN', 'BLH', 'CBA', 'CLR', 'CMW', 'DDB', 'DER', 'DIG', 'DRN', 'ETK', 'FGL', 'HR', 'HRG', 'JAD', 'JBE', 'JBN', 'JBT', 'JDO', 'JLB', 'JLP', 'JNC', 'KC', 'LLU', 'MGM', 'MIO', 'MKS', 'MSY', 'RB', 'RLR', 'RR1', 'RSH', 'RW', 'SEG', 'SG1', 'SME', 'TCR', 'VJ2', 'VJ4', 'VJ5', 'VJ6', 'VJ7', 'VJ8', 'WCP', 'WTN']

Original dataframe: 1510020 records
Filtered dataframe: 402672 records
Removed 1107348 records

Number of unique judges in filtered dataframe: 16
Verification successful: All judges with always_null_time have been removed


In [26]:
filtered_df

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload,family_size,hearing_datetime,assigned_hearing,next_assigned_hearing,time_to_next,assigned
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,832,1,2023-02-13 10:30:00,NaT,2022-12-12 13:00:00,27143340.0,0
2,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,DMP,0,1,2023-02-13 10:30:00,NaT,2023-07-20 13:00:00,46151340.0,0
5,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AWS,0,1,2023-02-13 10:30:00,NaT,2024-11-18 08:30:00,88211940.0,0
7,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,GYH,112,1,2023-02-13 10:30:00,NaT,2022-12-06 13:30:00,26626740.0,0
12,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,JGC,4,1,2023-02-13 10:30:00,NaT,2022-03-25 08:30:00,4490340.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1510006,53591844,13604711,14226348,AWS,2025-02-28,17:06:00,2025-02-28 17:06:00,2027-07-20,10:30:00,NES,0,3,2027-07-20 10:30:00,NaT,NaT,NaN,0
1510011,53591844,13604711,14226348,AWS,2025-02-28,17:06:00,2025-02-28 17:06:00,2027-07-20,10:30:00,MGC,210,3,2027-07-20 10:30:00,NaT,NaT,NaN,0
1510013,53591844,13604711,14226348,AWS,2025-02-28,17:06:00,2025-02-28 17:06:00,2027-07-20,10:30:00,KYT,3,3,2027-07-20 10:30:00,NaT,NaT,NaN,0
1510015,53591844,13604711,14226348,AWS,2025-02-28,17:06:00,2025-02-28 17:06:00,2027-07-20,10:30:00,KBR,1585,3,2027-07-20 10:30:00,NaT,NaT,NaN,0


In [27]:
percent_missing = filtered_df.isnull().sum() * 100 / len(filtered_df)
percent_missing

idnschedule               0.000000
idncase                   0.000000
idnproceeding             0.000000
assigned_judge            0.000000
scheduled_date            0.000000
scheduled_time            0.007947
scheduled_datetime        0.000000
hearing_date              0.000000
hearing_time              0.000000
judge                     0.000000
caseload                  0.000000
family_size               0.000000
hearing_datetime          0.000000
assigned_hearing         93.750000
next_assigned_hearing    14.780019
time_to_next             14.780019
assigned                  0.000000
dtype: float64

In [28]:
#filtered_df.loc[filtered_df['caseload'] == 0]
#zero_caseload_series = filtered_df.groupby('judge')['caseload'].apply(lambda x: (x==0).all())

In [29]:

# Create a clean dataframe for analysis by dropping rows with NaN values
regression_df = filtered_df.dropna(subset=['time_to_next', 'caseload', 'assigned'])

# Add a constant (intercept) to the model
X = sm.add_constant(regression_df[['time_to_next', 'caseload']])
y = regression_df['assigned']

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:               assigned   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     4067.
Date:                Wed, 23 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:52:35   Log-Likelihood:                -21555.
No. Observations:              343157   AIC:                         4.312e+04
Df Residuals:                  343154   BIC:                         4.315e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0024      0.001      2.471   

In [30]:
#regression_df.loc[regression_df['caseload'] != 0]
regression_df_no_0_caseload = regression_df.loc[regression_df['caseload'] != 0]

# Add a constant (intercept) to the model
X = sm.add_constant(regression_df_no_0_caseload[['time_to_next', 'caseload']])
y = regression_df_no_0_caseload['assigned']

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               assigned   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     2927.
Date:                Wed, 23 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:52:35   Log-Likelihood:                -32364.
No. Observations:              316679   AIC:                         6.473e+04
Df Residuals:                  316676   BIC:                         6.477e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0067      0.001      6.257   

In [31]:
regression_df['no_cases'] = (regression_df['caseload'] == 0).astype(int)
regression_df[['caseload', 'no_cases']]

/tmp/ipykernel_76054/412076632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regression_df['no_cases'] = (regression_df['caseload'] == 0).astype(int)


,caseload,no_cases
0,832,0
2,0,1
5,0,1
7,112,0
12,4,0
...,...,...
1509870,1461,0
1509880,209,0
1509910,210,0
1509934,1652,0


In [32]:
regression_df

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,caseload,family_size,hearing_datetime,assigned_hearing,next_assigned_hearing,time_to_next,assigned,no_cases
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,832,1,2023-02-13 10:30:00,NaT,2022-12-12 13:00:00,27143340.0,0,0
2,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,DMP,0,1,2023-02-13 10:30:00,NaT,2023-07-20 13:00:00,46151340.0,0,1
5,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AWS,0,1,2023-02-13 10:30:00,NaT,2024-11-18 08:30:00,88211940.0,0,1
7,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,GYH,112,1,2023-02-13 10:30:00,NaT,2022-12-06 13:30:00,26626740.0,0,0
12,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,JGC,4,1,2023-02-13 10:30:00,NaT,2022-03-25 08:30:00,4490340.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1509870,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,DAJ,1461,1,2025-05-01 10:30:00,2025-05-01 10:30:00,2025-05-01 10:30:00,5340300.0,1,0
1509880,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,MGC,209,1,2025-05-01 10:30:00,NaT,2025-11-20 13:00:00,22888500.0,0,0
1509910,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,MGC,210,1,2025-11-20 13:00:00,2025-11-20 13:00:00,2025-11-20 13:00:00,22885080.0,1,0
1509934,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,AWS,1652,1,2025-11-20 13:00:00,NaT,2027-07-20 10:30:00,75320880.0,0,0


In [33]:
# Add a constant (intercept) to the model
X = sm.add_constant(regression_df[['time_to_next', 'caseload', 'no_cases']])
y = regression_df['assigned']

# Fit the linear regression model
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:               assigned   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     2839.
Date:                Wed, 23 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:52:35   Log-Likelihood:                -21368.
No. Observations:              343157   AIC:                         4.274e+04
Df Residuals:                  343153   BIC:                         4.279e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0088      0.001      8.687   

In [34]:
dir(model)

['HC0_se',
 'HC1_se',
 'HC2_se',
 'HC3_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abat_diagonal',
 '_cache',
 '_data_attr',
 '_data_in_cache',
 '_get_robustcov_results',
 '_get_wald_nonlinear',
 '_is_nested',
 '_transform_predict_exog',
 '_use_t',
 '_wexog_singular_values',
 'aic',
 'bic',
 'bse',
 'centered_tss',
 'compare_f_test',
 'compare_lm_test',
 'compare_lr_test',
 'condition_number',
 'conf_int',
 'conf_int_el',
 'cov_HC0',
 'cov_HC1',
 'cov_HC2',
 'cov_HC3',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'diagn',
 'eigenvals',
 'el_test',
 'ess',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'get_influence',
 

In [35]:
regression_df.shape

(343157, 18)

In [36]:
def add_case_info(hearings_df, conn, postfix=POSTFIX):
    """
    Add case information to the scheduling records
    """
    case_table = f"foia_case_{postfix}"
    
    # Get unique case IDs
    case_ids = hearings_df['idncase'].unique().tolist()
    
    # Handle empty case_ids list
    if not case_ids:
        return hearings_df
    
    # Use placeholders for IN clause
    placeholders = ','.join(['%s'] * len(case_ids))
    
    # SQL to get case info with expanded fields
    query = f"""
    SELECT 
        idncase,
        nat,
        lang,
        e_28_date,
        casepriority_code,
        date_of_entry,
        c_birthdate,
        gender,
        lpr
    FROM {case_table}
    WHERE idncase IN ({placeholders})
    """
    
    # Execute query
    with conn.cursor() as cur:
        cur.execute(query, case_ids)
        columns = [desc[0] for desc in cur.description]
        results = cur.fetchall()
    
    # Convert to DataFrame
    case_df = pd.DataFrame(results, columns=columns)
    
    # Convert date columns
    date_cols = ['e_28_date', 'c_birthdate', 'date_of_entry', 'date_detained', 'date_released']
    for col in date_cols:
        if col in case_df.columns:
            case_df[col] = pd.to_datetime(case_df[col])
    
    # Merge with hearings
    result = hearings_df.merge(case_df, on='idncase', how='left')
    return result

with connect_to_db(db_params) as conn:
    edf = add_case_info(regression_df, conn)
edf



Database connection established successfully


/tmp/ipykernel_76054/2089072138.py:46: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  case_df[col] = pd.to_datetime(case_df[col])


,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,...,assigned,no_cases,nat,lang,e_28_date,casepriority_code,date_of_entry,c_birthdate,gender,lpr
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,...,0,0,NU,SP,2022-01-13 00:00:00.000,None,2013-04-25,1985-06-01,None,None
1,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,DMP,...,0,1,NU,SP,2022-01-13 00:00:00.000,None,2013-04-25,1985-06-01,None,None
2,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AWS,...,0,1,NU,SP,2022-01-13 00:00:00.000,None,2013-04-25,1985-06-01,None,None
3,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,GYH,...,0,0,NU,SP,2022-01-13 00:00:00.000,None,2013-04-25,1985-06-01,None,None
4,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,JGC,...,0,0,NU,SP,2022-01-13 00:00:00.000,None,2013-04-25,1985-06-01,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343152,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,DAJ,...,1,0,NU,SP,2022-11-17 18:18:00.260,None,2022-03-20,1970-12-01,F,None
343153,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,MGC,...,0,0,NU,SP,2022-11-17 18:18:00.260,None,2022-03-20,1970-12-01,F,None
343154,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,MGC,...,1,0,MX,ENG,2024-11-05 12:37:33.640,CR,NaT,1993-09-01,M,None
343155,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,AWS,...,0,0,MX,ENG,2024-11-05 12:37:33.640,CR,NaT,1993-09-01,M,None


In [37]:
edf['lawyer'] = edf.apply(
    lambda row: 1 if (row['e_28_date'] is not pd.NaT and 
                      row['e_28_date'] < row['scheduled_datetime']) else 0, 
    axis=1
)
edf[['e_28_date', 'scheduled_datetime', 'lawyer']]

,e_28_date,scheduled_datetime,lawyer
0,2022-01-13 00:00:00.000,2022-02-01 09:11:00,1
1,2022-01-13 00:00:00.000,2022-02-01 09:11:00,1
2,2022-01-13 00:00:00.000,2022-02-01 09:11:00,1
3,2022-01-13 00:00:00.000,2022-02-01 09:11:00,1
4,2022-01-13 00:00:00.000,2022-02-01 09:11:00,1
...,...,...,...
343152,2022-11-17 18:18:00.260,2025-02-28 15:05:00,1
343153,2022-11-17 18:18:00.260,2025-02-28 15:05:00,1
343154,2024-11-05 12:37:33.640,2025-02-28 16:02:00,1
343155,2024-11-05 12:37:33.640,2025-02-28 16:02:00,1


In [38]:
edf['nat'].value_counts()

nat
VE    66118
CU    54518
HO    52610
ES    40502
MX    30358
      ...  
AM       13
GY       13
SF       12
FO       12
KV       11
Name: count, Length: 115, dtype: int64

In [39]:
edf['lang'].value_counts()

lang
SP     296157
ENG     19765
MAN      3244
TU       2673
AR       2260
        ...  
SPS        11
BAM        11
SWA        11
ALB        11
MAR         9
Name: count, Length: 88, dtype: int64

In [40]:
edf['date_of_entry'].isnull().sum() * 100 / len(edf['date_of_entry'])

np.float64(25.941187269966807)

In [41]:
edf.columns

Index(['idnschedule', 'idncase', 'idnproceeding', 'assigned_judge',
       'scheduled_date', 'scheduled_time', 'scheduled_datetime',
       'hearing_date', 'hearing_time', 'judge', 'caseload', 'family_size',
       'hearing_datetime', 'assigned_hearing', 'next_assigned_hearing',
       'time_to_next', 'assigned', 'no_cases', 'nat', 'lang', 'e_28_date',
       'casepriority_code', 'date_of_entry', 'c_birthdate', 'gender', 'lpr',
       'lawyer'],
      dtype='object')

In [42]:
def add_proceeding_info(hearings_df, conn, postfix=POSTFIX):
    """
    Add proceeding information to the scheduling records
    """
    # Get unique case IDs
    proc_ids = hearings_df['idnproceeding'].unique().tolist()
    
    # Handle empty case_ids list
    if not proc_ids:
        return hearings_df
    
    # Use placeholders for IN clause
    placeholders = ','.join(['%s'] * len(proc_ids))
    
    # SQL to get case info with expanded fields
    query = f"""
    SELECT 
        idnproceeding,
        crim_ind,
        casetype,
        custody,
        osc_date
    FROM {proceeding_table}
    WHERE idnproceeding IN ({placeholders})
    """
    
    # Execute query
    with conn.cursor() as cur:
        cur.execute(query, proc_ids)
        columns = [desc[0] for desc in cur.description]
        results = cur.fetchall()
    
    # Convert to DataFrame
    proc_df = pd.DataFrame(results, columns=columns)
    
    # Merge with hearings
    result = hearings_df.merge(proc_df, on='idnproceeding', how='left')
    return result

with connect_to_db(db_params) as conn:
    xdf = add_proceeding_info(edf, conn)


Database connection established successfully


In [43]:
xdf

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,...,casepriority_code,date_of_entry,c_birthdate,gender,lpr,lawyer,crim_ind,casetype,custody,osc_date
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,...,None,2013-04-25,1985-06-01,None,None,1,N,RMV,N,2013-05-30
1,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,DMP,...,None,2013-04-25,1985-06-01,None,None,1,N,RMV,N,2013-05-30
2,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AWS,...,None,2013-04-25,1985-06-01,None,None,1,N,RMV,N,2013-05-30
3,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,GYH,...,None,2013-04-25,1985-06-01,None,None,1,N,RMV,N,2013-05-30
4,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,JGC,...,None,2013-04-25,1985-06-01,None,None,1,N,RMV,N,2013-05-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343152,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,DAJ,...,None,2022-03-20,1970-12-01,F,None,1,N,RMV,N,2022-03-22
343153,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,MGC,...,None,2022-03-20,1970-12-01,F,None,1,N,RMV,N,2022-03-22
343154,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,MGC,...,CR,NaT,1993-09-01,M,None,1,N,RMV,R,2024-10-27
343155,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,AWS,...,CR,NaT,1993-09-01,M,None,1,N,RMV,R,2024-10-27


In [44]:
xdf.columns

Index(['idnschedule', 'idncase', 'idnproceeding', 'assigned_judge',
       'scheduled_date', 'scheduled_time', 'scheduled_datetime',
       'hearing_date', 'hearing_time', 'judge', 'caseload', 'family_size',
       'hearing_datetime', 'assigned_hearing', 'next_assigned_hearing',
       'time_to_next', 'assigned', 'no_cases', 'nat', 'lang', 'e_28_date',
       'casepriority_code', 'date_of_entry', 'c_birthdate', 'gender', 'lpr',
       'lawyer', 'crim_ind', 'casetype', 'custody', 'osc_date'],
      dtype='object')

In [45]:
wdf = pd.get_dummies(xdf, columns=['casetype','custody', 'casepriority_code', 'crim_ind'], prefix=['casetype','custody', 'casepriority_', 'crim_'], dtype=int)

In [46]:
diff = (wdf['scheduled_datetime'] - wdf['osc_date'])
wdf['length_of_proceedings'] = diff.apply(lambda x: x.days)
wdf

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,...,custody_N,custody_R,casepriority__AWC/ATD,casepriority__AWC/D,casepriority__CR,casepriority__RBC/D,casepriority__UC,crim__N,crim__Y,length_of_proceedings
0,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AAJ,...,1,0,0,0,0,0,0,1,0,3169
1,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,DMP,...,1,0,0,0,0,0,0,1,0,3169
2,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,AWS,...,1,0,0,0,0,0,0,1,0,3169
3,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,GYH,...,1,0,0,0,0,0,0,1,0,3169
4,40518386,7356844,9755557,MGC,2022-02-01,09:11:00,2022-02-01 09:11:00,2023-02-13,10:30:00,JGC,...,1,0,0,0,0,0,0,1,0,3169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343152,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,DAJ,...,1,0,0,0,0,0,0,1,0,1074
343153,53589371,10480951,11575807,DAJ,2025-02-28,15:05:00,2025-02-28 15:05:00,2025-05-01,10:30:00,MGC,...,1,0,0,0,0,0,0,1,0,1074
343154,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,MGC,...,0,1,0,0,1,0,0,1,0,124
343155,53590365,14635473,15970447,MGC,2025-02-28,16:02:00,2025-02-28 16:02:00,2025-11-20,13:00:00,AWS,...,0,1,0,0,1,0,0,1,0,124


In [47]:
counts = wdf['lang'].value_counts()
top5 = counts.nlargest(5).index  # Get the names of the top 5 languages

# Create a new column that labels languages not in the top 5 as "other"
wdf['lang_top5_or_other'] = np.where(wdf['lang'].isin(top5), wdf['lang'], 'other')

# Create dummy variables for the new column. This will create one dummy per top language and one for "other"
dummies = pd.get_dummies(wdf['lang_top5_or_other'], prefix='lang', dtype=int)

# Optionally, join the dummies back to your dataframe
wdf = pd.concat([wdf, dummies], axis=1)

# Optionally, if you don't need the helper column any more:
wdf.drop(columns='lang_top5_or_other', inplace=True)
wdf[['lang', 'lang_SP']]

,lang,lang_SP
0,SP,1
1,SP,1
2,SP,1
3,SP,1
4,SP,1
...,...,...
343152,SP,1
343153,SP,1
343154,ENG,0
343155,ENG,0


In [48]:
counts = wdf['nat'].value_counts()
top5 = counts.nlargest(5).index  # Get the names of the top 5 languages

# Create a new column that labels languages not in the top 5 as "other"
wdf['nat_top5_or_other'] = np.where(wdf['nat'].isin(top5), wdf['nat'], 'other')

# Create dummy variables for the new column. This will create one dummy per top language and one for "other"
dummies = pd.get_dummies(wdf['nat_top5_or_other'], prefix='nat', dtype=int)

# Optionally, join the dummies back to your dataframe
wdf = pd.concat([wdf, dummies], axis=1)

# Optionally, if you don't need the helper column any more:
wdf.drop(columns='nat_top5_or_other', inplace=True)
wdf[['nat', 'nat_VE']]

,nat,nat_VE
0,NU,0
1,NU,0
2,NU,0
3,NU,0
4,NU,0
...,...,...
343152,NU,0
343153,NU,0
343154,MX,0
343155,MX,0


In [49]:
percent_missing = wdf['c_birthdate'].isnull().sum() * 100 / len(wdf['c_birthdate'])
percent_missing

np.float64(21.296957369367373)

In [50]:
# First, sort the dataframe by case and scheduled_datetime
wdf = wdf.sort_values(['idncase', 'scheduled_datetime'])

# Identify which judge was actually assigned in each group
assigned_judges = wdf[wdf['assigned'] == 1].groupby(['idncase', 'scheduled_datetime'])['assigned_judge'].first().reset_index()
assigned_judges.columns = ['idncase', 'scheduled_datetime', 'previous_assigned_judge']

# Shift the scheduled_datetime to get the next one
assigned_judges['next_datetime'] = assigned_judges.groupby('idncase')['scheduled_datetime'].shift(-1)

# Merge this back to get the previously assigned judge for each next group
wdf = wdf.merge(
    assigned_judges[['idncase', 'previous_assigned_judge', 'next_datetime']],
    left_on=['idncase', 'scheduled_datetime'],
    right_on=['idncase', 'next_datetime'],
    how='left'
)

# Create the previously_assigned indicator
wdf['previously_assigned'] = (wdf['assigned_judge'] == wdf['previous_assigned_judge']).astype(int)

# Clean up - drop the temporary column and handle NaNs
wdf = wdf.drop('next_datetime', axis=1)
wdf['previously_assigned'] = wdf['previously_assigned'].fillna(0)

In [51]:
wdf

,idnschedule,idncase,idnproceeding,assigned_judge,scheduled_date,scheduled_time,scheduled_datetime,hearing_date,hearing_time,judge,...,lang_TU,lang_other,nat_CU,nat_ES,nat_HO,nat_MX,nat_VE,nat_other,previous_assigned_judge,previously_assigned
0,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,NES,...,0,1,0,0,0,0,0,1,NaN,0
1,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,DAJ,...,0,1,0,0,0,0,0,1,NaN,0
2,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,AAJ,...,0,1,0,0,0,0,0,1,NaN,0
3,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,AWS,...,0,1,0,0,0,0,0,1,NaN,0
4,41417564,2265996,10415583,MGC,2022-05-03,16:25:00,2022-05-03 16:25:00,2023-06-15,13:00:00,JDA,...,0,1,0,0,0,0,0,1,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343152,53289036,14873189,15900528,AAJ,2025-02-03,09:47:00,2025-02-03 09:47:00,2025-02-04,08:00:00,CMS,...,0,0,0,1,0,0,0,0,NaN,0
343153,53289036,14873189,15900528,AAJ,2025-02-03,09:47:00,2025-02-03 09:47:00,2025-02-04,08:00:00,MGC,...,0,0,0,1,0,0,0,0,NaN,0
343154,53289036,14873189,15900528,AAJ,2025-02-03,09:47:00,2025-02-03 09:47:00,2025-02-04,08:00:00,DMP,...,0,0,0,1,0,0,0,0,NaN,0
343155,53289036,14873189,15900528,AAJ,2025-02-03,09:47:00,2025-02-03 09:47:00,2025-02-04,08:00:00,KBR,...,0,0,0,1,0,0,0,0,NaN,0


In [52]:
#wdf['family_size'].value_counts()
wdf['family'] = (wdf['family_size'] > 1).astype(int)
wdf[['family', 'family_size']]

,family,family_size
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
343152,1,4
343153,1,4
343154,1,4
343155,1,4


In [53]:
def add_schedule_info(hearings_df, conn, postfix=POSTFIX):
    """
    Add proceeding information to the scheduling records
    """
    # Get unique case IDs
    case_ids = hearings_df['idncase'].unique().tolist()
    
    # Handle empty case_ids list
    if not case_ids:
        return hearings_df
    
    # Use placeholders for IN clause
    placeholders = ','.join(['%s'] * len(case_ids))
    
    # SQL query to get scheduling info only for the specified cases
    query = f"""
    SELECT 
        idncase,
        json_agg(
            json_build_object(
                'idnschedule', idnschedule,
                'ij_code', ij_code,
                'adj_date', adj_date,
                'adj_time', adj_time_start,
                'adj_rsn', adj_rsn,
                'cal_type', cal_type,
                'schedule_type', schedule_type,
                'input_date', input_date,
                'hearing_loc_code', hearing_loc_code,
                'base_city_code', base_city_code
            ) ORDER BY input_date, adj_date, adj_time_start
        ) AS schedules
    FROM foia_schedule_03_25 
    WHERE idncase IN ({placeholders})
    AND base_city_code = 'HSG'
    GROUP BY idncase
    """
    
    # Execute query
    with conn.cursor() as cur:
        cur.execute(query, case_ids)
        columns = [desc[0] for desc in cur.description]
        results = cur.fetchall()
    
    return results

with connect_to_db(db_params) as conn:
    results = add_schedule_info(wdf, conn)

Database connection established successfully


In [54]:
for_df = []
for res in results:
    idncase = res[0]
    for rec in res[1]:
        tup = (idncase,) + tuple(rec.values())
        for_df.append(tup)

columns = ['idncase'] + list(results[0][1][0].keys())
sched_recs = pd.DataFrame(for_df, columns=columns)
sched_recs['adj_rsn'].value_counts()

        

adj_rsn
59    10605
9V    10179
17     6577
01     6438
8B     5486
      ...  
31        1
60        1
24        1
4B        1
1B        1
Name: count, Length: 71, dtype: int64

In [55]:
adj_dict = LOOKUPS["tblAdjournmentCodes.json"]

In [56]:
sched_recs['adj_decode'] = sched_recs['adj_rsn'].apply(lambda x: adj_dict.get(x))
sched_recs

,idncase,idnschedule,ij_code,adj_date,adj_time,adj_rsn,cal_type,schedule_type,input_date,hearing_loc_code,base_city_code,adj_decode
0,2265996,40480134,MGC,2022-05-03T00:00:00,13:00:00,17,M,MR,2022-01-27T00:00:00,HSG,HSG,MC TO IC--MERITS HEARING
1,2265996,41417564,MGC,2023-06-15T00:00:00,13:00:00,4A,I,II,2022-05-03T00:00:00,HSG,HSG,TECHNICAL MALFUNCTION (NOT VIDEO)
2,2265996,45465829,MGC,2023-07-11T00:00:00,13:30:00,12,M,MS,2023-06-20T00:00:00,HSG,HSG,OTHER ALIEN/ALIEN'S ATTY/REPRESENTATIVE REQUEST
3,2265996,45609073,MGC,2023-08-15T00:00:00,13:30:00,12,M,MS,2023-07-03T00:00:00,HSG,HSG,OTHER ALIEN/ALIEN'S ATTY/REPRESENTATIVE REQUEST
4,2265996,45896341,NES,2023-10-10T00:00:00,13:30:00,02,M,MS,2023-07-26T00:00:00,HSG,HSG,PREPARATION--ALIEN/ATTORNEY/REPRESENTATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...
76562,14850222,53118249,JJC,2025-01-21T00:00:00,08:00:00,59,M,MC,2025-01-16T11:34:47.82,FHG,HSG,COURT CLOSURE
76563,14850222,53149277,JJC,2025-01-24T00:00:00,08:00:00,8B,M,MC,2025-01-21T00:00:00,FHG,HSG,IJ COMPLETION AT HEARING
76564,14850222,53149064,JJC,2025-01-24T00:00:00,08:00:00,99,I,RR,2025-01-21T00:00:00,FHG,HSG,Data Entry Error
76565,14873189,53278408,AAJ,2025-02-03T00:00:00,08:00:00,33,M,MC,2025-01-31T11:01:02.697,FHG,HSG,"INTERPRETER ORDERED, BUT FTA"


In [57]:
# ijs = wdf['assigned_judge'].value_counts().index.to_list()
# ijs

In [58]:
# # First, identify the cases that meet both conditions
# m_condition = (sched_recs['cal_type'] == 'M') & (sched_recs['ij_code'].isin(ijs))
# i_condition = (sched_recs['cal_type'] == 'I') & (sched_recs['ij_code'].isin(ijs))

# # Get idncase values that have at least one M and one I hearing with qualified ij_codes
# cases_with_m = set(sched_recs[m_condition]['idncase'].unique())
# cases_with_i = set(sched_recs[i_condition]['idncase'].unique())
# qualifying_cases = cases_with_m.intersection(cases_with_i)

# # Get all records for these qualifying cases (unfiltered groups)
# result = sched_recs[sched_recs['idncase'].isin(qualifying_cases)]
# result.to_csv('ij_transfer_recs.csv')

In [60]:
adjournment_codes = {
    # Court/Administrative closures
    "59": {
        "description": "COURT CLOSURE",
        "never_occurred": True,
        "category": "Court/Administrative closure"
    },
    "9V": {
        "description": "VACATED MASTER CALENDAR",
        "never_occurred": True,
        "category": "Court/Administrative closure"
    },
    
    # Judge unavailability
    "34": {
        "description": "IJ LEAVE",
        "never_occurred": True,
        "category": "Judge unavailability"
    },
    # "64": {
    #     "description": "IJ REASSIGNMENT",
    #     "never_occurred": True,
    #     "category": "Judge unavailability"
    # },
    # "8A": {
    #     "description": "IJ COMPLETION PRIOR TO HEARING",
    #     "never_occurred": True,
    #     "category": "Judge unavailability"
    # },
    # "3A": {
    #     "description": "IJ DETAIL (VOLUNTEER)",
    #     "never_occurred": True,
    #     "category": "Judge unavailability"
    # },
    
    # Technical issues
    "4A": {
        "description": "TECHNICAL MALFUNCTION (NOT VIDEO)",
        "never_occurred": True,
        "category": "Technical issues"
    },
    "4E": {
        "description": "ROP MISSING",
        "never_occurred": True,
        "category": "Technical issues"
    },
    
    # Administrative errors
    "99": {
        "description": "DATA ENTRY ERROR",
        "never_occurred": True,
        "category": "Administrative error"
    },
    
    # Docket management
    "9C": {
        "description": "DOCKET MANAGMENT CASE MOVED OFF CALENDAR",
        "never_occurred": True,
        "category": "Docket management"
    },
    "9B": {
        "description": "DOCKET MANAGEMENT (ADVANCE HEARING)",
        "never_occurred": True,
        "category": "Docket management"
    },
    "9A": {
        "description": "DOCKET MANAGEMENT (POSTPONE HEARING)",
        "never_occurred": False,
        "category": "Docket management"
    },
    "25": {
        "description": "TO ALLOW FOR SCHEDULING OF PRIORITY CASE",
        "never_occurred": True,
        "category": "Docket management"
    },
    
    # Case transfers/status changes
    # "PD": {
    #     "description": "PROSECUTORIAL DISCRETION CONSIDERATION",
    #     "never_occurred": True,
    #     "category": "Case status change"
    # },
    # "1A": {
    #     "description": "CASE TRANSFERRED FROM NON-DETAINED TO DETAINED DOCKET",
    #     "never_occurred": True,
    #     "category": "Case transfer"
    # },
    
    # Schedule changes
    "5A": {
        "description": "HEARING ADVANCED BY MOTION",
        "never_occurred": True,
        "category": "Schedule change"
    },
    
    # Hearings that did occur or potentially occurred
    "17": {
        "description": "MC TO IC--MERITS HEARING",
        "never_occurred": False,
        "category": "Procedural advancement"
    },
    "8B": {
        "description": "IJ COMPLETION AT HEARING",
        "never_occurred": False,
        "category": "Hearing completed"
    },
    "TQ": {
        "description": "TRIAL QUEUE",
        "never_occurred": False,
        "category": "Procedural advancement"
    },
    "RD": {
        "description": "RESERVED SUSPENSION DECISION",
        "never_occurred": False,
        "category": "Procedural advancement"
    },
    "RR": {
        "description": "RESERVED DECISION",
        "never_occurred": False,
        "category": "Procedural advancement"
    },
    
    # Alien/Attorney related
    "01": {
        "description": "ALIEN TO SEEK REPRESENTATION",
        "never_occurred": False,
        "category": "Alien/Attorney request"
    },
    "02": {
        "description": "PREPARATION--ALIEN/ATTORNEY/REPRESENTATIVE",
        "never_occurred": False,
        "category": "Alien/Attorney request"
    },
    "11": {
        "description": "OTHER NO-SHOW BY ALIEN/ALIEN'S ATTORNEY OR REP",
        "never_occurred": False,
        "category": "Alien/Attorney absence"
    },
    "12": {
        "description": "OTHER ALIEN/ALIEN'S ATTY/REPRESENTATIVE REQUEST",
        "never_occurred": False,
        "category": "Alien/Attorney request"
    },
    "38": {
        "description": "ILLNESS OF ALIEN/ATTY REP/WITNESS",
        "never_occurred": False,
        "category": "Alien/Attorney absence"
    },
    "36": {
        "description": "ALIEN DELAYED RECORDS/FINGERPRINT CHECK",
        "never_occurred": False,
        "category": "Alien/Attorney request"
    },
    "54": {
        "description": "ALIEN CLAIM TO U.S. CITIZENSHIP",
        "never_occurred": False,
        "category": "Procedural issue"
    },
    
    # DHS related
    "03": {
        "description": "PREPARATION--DHS",
        "never_occurred": False,
        "category": "DHS request"
    },
    "04": {
        "description": "DHS OR DHS ADMIN FILE UNAVAILABLE FOR HEARING",
        "never_occurred": False,
        "category": "DHS issue"
    },
    "7A": {
        "description": "DHS APPLICATION PROCESS - ALIEN INITIATED",
        "never_occurred": False,
        "category": "DHS process"
    },
    "37": {
        "description": "DHS INVESTIGATION",
        "never_occurred": False,
        "category": "DHS process"
    },
    
    # Other procedural
    "13": {
        "description": "INSUFFICIENT TIME TO COMPLETE HEARING",
        "never_occurred": False,
        "category": "Procedural issue"
    },
    "30": {
        "description": "CONSOLIDATION WITH FAMILY MEMBERS",
        "never_occurred": False,
        "category": "Procedural advancement"
    },
    "RO": {
        "description": "ATTORNEY/REP AND DHS TO RESPOND TO RESET ORDER",
        "never_occurred": False,
        "category": "Procedural advancement"
    }
}

# Usage example:
def check_never_occurred(adj_code):
    if adj_code in adjournment_codes:
        return adjournment_codes[adj_code]["never_occurred"]
    return False  # Default if code not found

def get_category(adj_code):
    if adj_code in adjournment_codes:
        return adjournment_codes[adj_code]["category"]
    return "Unknown code"

def get_description(adj_code):
    if adj_code in adjournment_codes:
        return adjournment_codes[adj_code]["description"]
    return "Unknown adjournment code"

In [61]:
# sched_recs = sched_recs[~sched_recs['adj_rsn'].apply(lambda x: check_never_occurred(x))]

In [62]:
set(wdf['idnschedule'].unique()).issubset(set(sched_recs['idnschedule']))

True

In [63]:
def check_judge_reassignment(wdf, sched_recs):
    # Get unique idnschedules from wdf
    unique_schedules = wdf['idnschedule'].unique()
    
    # Create a function to check for previous judge assignments
    def check_previous_judges(idnschedule):
        # Check if the idnschedule exists in sched_recs
        if idnschedule not in sched_recs['idnschedule'].values:
            return 0  # Schedule not found in sched_recs, cannot determine reassignment
        
        # Get the current record
        current_record = sched_recs[sched_recs['idnschedule'] == idnschedule].iloc[0]
        current_case = current_record['idncase']
        current_judge = current_record['ij_code']
        current_input_date = current_record['input_date']
        
        # Get all records for this case and sort by input_date
        case_records = sched_recs[sched_recs['idncase'] == current_case].copy()
        case_records['input_date'] = case_records['input_date']
        case_records = case_records.sort_values('input_date')
        
        # Find records that came before the current record
        previous_records = case_records[case_records['input_date'] < current_input_date]
        
        # Check if any previous records had a different judge
        if len(previous_records) > 0:
            previous_judges = previous_records['ij_code'].unique()
            return 1 if any(judge != current_judge for judge in previous_judges) else 0
        else:
            return 0  # No previous records
    
    # Create a dictionary to store results
    results = {}
    
    # Apply the function to each unique idnschedule
    for idnschedule in unique_schedules:
        results[idnschedule] = check_previous_judges(idnschedule)
    
    # Create a Series for mappingreghdfe assigned caseload time_to_next previously_assigned no_cases length_of_proceedings custody_d casepriority__uc family, absorb(scheduled_month_year);

    reassignment_map = pd.Series(results)
    
    # Add the column directly to wdf
    wdf['ij_throughout'] = wdf['idnschedule'].map(reassignment_map).fillna(0).astype(int)
    
    return wdf

# Apply the function
wdf_result = check_judge_reassignment(wdf, sched_recs)


In [64]:
wdf_result = wdf_result.sort_values(by='scheduled_datetime').reset_index(drop=True)
wdf_result.to_csv('check_prev_ij_assignment.csv')

In [65]:
wdf = wdf_result
wdf.columns

Index(['idnschedule', 'idncase', 'idnproceeding', 'assigned_judge',
       'scheduled_date', 'scheduled_time', 'scheduled_datetime',
       'hearing_date', 'hearing_time', 'judge', 'caseload', 'family_size',
       'hearing_datetime', 'assigned_hearing', 'next_assigned_hearing',
       'time_to_next', 'assigned', 'no_cases', 'nat', 'lang', 'e_28_date',
       'date_of_entry', 'c_birthdate', 'gender', 'lpr', 'lawyer', 'osc_date',
       'casetype_AOC', 'casetype_CFR', 'casetype_DEP', 'casetype_REC',
       'casetype_RFR', 'casetype_RMV', 'casetype_WHO', 'custody_D',
       'custody_N', 'custody_R', 'casepriority__AWC/ATD',
       'casepriority__AWC/D', 'casepriority__CR', 'casepriority__RBC/D',
       'casepriority__UC', 'crim__N', 'crim__Y', 'length_of_proceedings',
       'lang_AR', 'lang_ENG', 'lang_MAN', 'lang_SP', 'lang_TU', 'lang_other',
       'nat_CU', 'nat_ES', 'nat_HO', 'nat_MX', 'nat_VE', 'nat_other',
       'previous_assigned_judge', 'previously_assigned', 'family',
       '

In [66]:
# wdf.to_csv('hsg_scheduling_data_2022_2025.csv')

In [67]:
percent_missing = wdf.isnull().sum() * 100 / len(wdf)
percent_missing

idnschedule                 0.000000
idncase                     0.000000
idnproceeding               0.000000
assigned_judge              0.000000
scheduled_date              0.000000
                             ...    
nat_other                   0.000000
previous_assigned_judge    73.420038
previously_assigned         0.000000
family                      0.000000
ij_throughout               0.000000
Length: 61, dtype: float64

In [68]:
# # Add a constant (intercept) to the model
# null = ['time_to_next', 'caseload', 'no_cases']
# enhanced_null_prev_assigned = null + ['previously_assigned']
# enhanced_null_prev_assigned_ij_throughout = null + ['previously_assigned', 'ij_throughout']
# priority_dockets = ['length_of_proceedings', 'custody_N', 'casepriority__UC', 'family']

# # Categorial dummies
# columns = list(wdf.columns)
# lang = [col for col in columns if 'lang_' in col]
# nat = [col for col in columns if 'nat_' in col]
# casetype = [col for col in columns if 'casetype_' in col]
# custody = [col for col in columns if 'custody_' in col]

# model_variables = { "Null": null, 
#           "Enhanced Null": enhanced_null_prev_assigned, 
#           "Enhanced Null IJ Throughout": enhanced_null_prev_assigned_ij_throughout, 
#           "Null and Priority Dockets": null + priority_dockets,
#           "Enhanced Null and Priority Dockets": enhanced_null_prev_assigned + priority_dockets,
#           "Null and Custody" : null + priority_dockets + custody,
#           "Enhanced Null and Custody" : enhanced_null_prev_assigned + priority_dockets + custody,
#           "Kitchen Sink": enhanced_null_prev_assigned + priority_dockets + lang + nat + casetype + custody + ['lawyer']
#          }
# models = {}

# for title, x_variables in model_variables.items():
#     X = sm.add_constant(wdf[list(set(x_variables))])
#     y = wdf['assigned']
#     model = sm.OLS(y, X).fit()
#     models[title] = model
#     print(f"\n{'='*25}  {title.upper()}  {'='*25}\n")
#     print(model.summary())



In [69]:
# Add a constant (intercept) to the model
null = ['time_to_next', 'caseload', 'no_cases']
enhanced_null_prev_assigned = null + ['previously_assigned']
enhanced_null_prev_assigned_ij_throughout = null + ['previously_assigned', 'ij_throughout']
priority_dockets = ['length_of_proceedings', 'custody_N', 'casepriority__UC', 'family']

# Categorial dummies
columns = list(wdf.columns)
lang = [col for col in columns if 'lang_' in col]
nat = [col for col in columns if 'nat_' in col]
casetype = [col for col in columns if 'casetype_' in col]
custody = [col for col in columns if 'custody_' in col]

# Add new combinations
model_variables = { 
    "Null": null, 
    "Enhanced Null": enhanced_null_prev_assigned, 
    "Enhanced Null IJ Throughout": enhanced_null_prev_assigned_ij_throughout, 
    "Null and Priority Dockets": null + priority_dockets,
    "Enhanced Null and Priority Dockets": enhanced_null_prev_assigned + priority_dockets,
    "Enhanced Null IJ Throughout and Priority": enhanced_null_prev_assigned_ij_throughout + priority_dockets,
    "Null and Language": null + lang,
    "Null and Nationality": null + nat,
    "Null and Case Type": null + casetype,
    "Null and Custody": null + custody,
    "Enhanced Null and Custody": enhanced_null_prev_assigned + custody,
    "Enhanced Null and Language": enhanced_null_prev_assigned + lang,
    "Enhanced Null and Nationality": enhanced_null_prev_assigned + nat,
    "Enhanced Null and Case Type": enhanced_null_prev_assigned + casetype,
    "Enhanced Null with Priority and Language": enhanced_null_prev_assigned + priority_dockets + lang,
    "Enhanced Null with Priority and Nationality": enhanced_null_prev_assigned + priority_dockets + nat,
    "Enhanced Null with Priority and Case Type": enhanced_null_prev_assigned + priority_dockets + casetype,
    "Enhanced Null with Priority and Custody": enhanced_null_prev_assigned + priority_dockets + custody,
    "Full Model (No Lawyer)": enhanced_null_prev_assigned_ij_throughout + priority_dockets + lang + nat + casetype + custody,
    "Kitchen Sink": enhanced_null_prev_assigned_ij_throughout + priority_dockets + lang + nat + casetype + custody + ['lawyer']
}

# Run the models as before
models = {}
for title, x_variables in model_variables.items():
    X = sm.add_constant(wdf[list(set(x_variables))])
    y = wdf['assigned']
    model = sm.OLS(y, X).fit()
    models[title] = model
    print(f"\n{'='*25}  {title.upper()}  {'='*25}\n")
    print(model.summary())


=========================  NULL  =========================

                            OLS Regression Results                            
Dep. Variable:               assigned   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     2839.
Date:                Wed, 23 Apr 2025   Prob (F-statistic):               0.00
Time:                        23:54:49   Log-Likelihood:                -21368.
No. Observations:              343157   AIC:                         4.274e+04
Df Residuals:                  343153   BIC:                         4.279e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

In [70]:
wdf.to_csv('hsg_schedules_2022_2025.csv')

In [71]:
# Create a table to track variable significance across models
import pandas as pd
import numpy as np

# Get all unique variables across all models
all_variables = set()
for model in models.values():
    all_variables.update(model.params.index)

# Remove the constant term
if 'const' in all_variables:
    all_variables.remove('const')

# Create a DataFrame to store significance levels
significance_df = pd.DataFrame(index=sorted(all_variables), columns=models.keys())

# Fill the DataFrame with significance indicators
for model_name, model in models.items():
    for var in all_variables:
        if var in model.params.index:
            p_value = model.pvalues[var]
            if p_value < 0.001:
                significance_df.loc[var, model_name] = '***'  # p < 0.001
            elif p_value < 0.01:
                significance_df.loc[var, model_name] = '**'   # p < 0.01
            elif p_value < 0.05:
                significance_df.loc[var, model_name] = '*'    # p < 0.05
            else:
                significance_df.loc[var, model_name] = ''     # not significant
        else:
            significance_df.loc[var, model_name] = 'N/A'      # variable not in model

# Calculate how often variables are significant across models
sig_counts = pd.DataFrame(index=significance_df.index)
sig_counts['significant_count'] = significance_df.apply(lambda x: sum([1 for val in x if val in ['*', '**', '***']]), axis=1)
sig_counts['models_included'] = significance_df.apply(lambda x: sum([1 for val in x if val != 'N/A']), axis=1)
sig_counts['significant_ratio'] = sig_counts['significant_count'] / sig_counts['models_included']

# Sort by the ratio of models in which the variable is significant
sig_counts = sig_counts.sort_values('significant_ratio', ascending=False)

# Display the top 20 most consistently significant variables
print("Most Consistently Significant Variables:")
print(sig_counts.head(20))

Most Consistently Significant Variables:
                       significant_count  models_included  significant_ratio
caseload                              20               20           1.000000
casepriority__UC                       9                9           1.000000
custody_R                              5                5           1.000000
previously_assigned                   14               14           1.000000
time_to_next                          20               20           1.000000
no_cases                              20               20           1.000000
length_of_proceedings                  9                9           1.000000
casetype_CFR                           3                5           0.600000
nat_HO                                 3                5           0.600000
custody_N                              6               11           0.545455
nat_CU                                 2                5           0.400000
nat_ES                             

# Time Fixed Effects
- effectively implemetned them by choosing the sample, but I could still throw
- Year month fixed effects, doesn't matter what label you assign.
- Use fixed effects estimator.

conditional on teh things I think matter and the statistical procedure matters, it's still the case that the assignment process has a lot of variation init. The question is whetehr that remaining variation is random. Put in variables that I think are part of the assignment process, at the end of the day some of these variables matter a lot, but a lot of the assignment is random. There is an assignment mechanism, but it's still a bunch of judges, and so a coin is flipped. 

# Ideas for Next Week
- people are looking at data from this context assuming that judges are randomly assigned and using data from those outcomes
- one way to write this paper is to interrogate the degree to which those papers are actually using random variation
- Hungry judges - whether cases heard later in the day were less lenient than those further from lunch time. When cases during the day was random, they didn't have an institutional statement that the cases was random. A bunch of cases came out afterwards saying that actually if you look it's not random about whenc ases are assigned. IT seems that harder cases were scheduled for later in the day.
- Do work to recover what is the plausibly random component of assignment and this can be a sprinboard for future work using the possibly random component of assignment.
- Tell us about what goes wrong if you use the full variation versus the plausibly random variation.
- Maybe it is the case if the full variation ends up picking up a lot of national origin variation, whereas the conditional variation is not picking up a lot of national origin variation.
- Back to the original thing, let's go and try to estimate judge leniency. Try to do that in a way that is purged of endogenous variation of whcih cases are heard by which judge.
- Judges are hearing lots of cases, some of the cases they are hearing are under their control or under the system's control. If a judge really wanted to hear some cases for ac ertain group they know is coming in the system soon, they could clear their docket so they have a fast time to next case. That's the selection that we would worry about. Because that judge is more or less favorable to that identifiable group.
- Judge leniency estimates purged of any judge selection.
- Segmenting the variation, kicking out the data where the judge has selected their case to some degree. You're just keeping the data where this coin flip has happened. Calculate variation and then do coin flip variation and you see that they are pretty similar. Good news you don't have to do that work to get reasonable estimates of judge leniency.
- Could be the case that even though its only 2% of the variation it's a really important 2% of the variation.
- People who work on this area who would be thrilled to work on a paper that's using this data.
- Fourth option is that this is step one to actually going and using this to study some downstream or later in life study. Isolating exogenous poriton of assignment mechanism so now I can estimate these judge leniency factors. These judge leniency factors are the instruments that go into a random judges IV first stage. First stage is the first step for a two stage least squares. Use this an instrument for immigraiton into a certain area.
- Reduced form applied micro loves an instrument.
- Causal random forest is really going to test, does other stuff matter. Let's test balance on the observables.
- What other X's go in that estimating equation that I want to condition on.
- Put in or out X variables in the value added estimating equation. Only keeping the subset of stuff they were assigned to that was plausibly exogenously assigned. 

In [72]:
len(wdf['idncase'].unique())

18109

In [73]:
columns

['idnschedule',
 'idncase',
 'idnproceeding',
 'assigned_judge',
 'scheduled_date',
 'scheduled_time',
 'scheduled_datetime',
 'hearing_date',
 'hearing_time',
 'judge',
 'caseload',
 'family_size',
 'hearing_datetime',
 'assigned_hearing',
 'next_assigned_hearing',
 'time_to_next',
 'assigned',
 'no_cases',
 'nat',
 'lang',
 'e_28_date',
 'date_of_entry',
 'c_birthdate',
 'gender',
 'lpr',
 'lawyer',
 'osc_date',
 'casetype_AOC',
 'casetype_CFR',
 'casetype_DEP',
 'casetype_REC',
 'casetype_RFR',
 'casetype_RMV',
 'casetype_WHO',
 'custody_D',
 'custody_N',
 'custody_R',
 'casepriority__AWC/ATD',
 'casepriority__AWC/D',
 'casepriority__CR',
 'casepriority__RBC/D',
 'casepriority__UC',
 'crim__N',
 'crim__Y',
 'length_of_proceedings',
 'lang_AR',
 'lang_ENG',
 'lang_MAN',
 'lang_SP',
 'lang_TU',
 'lang_other',
 'nat_CU',
 'nat_ES',
 'nat_HO',
 'nat_MX',
 'nat_VE',
 'nat_other',
 'previous_assigned_judge',
 'previously_assigned',
 'family',
 'ij_throughout']

In [74]:
cols_to_keep = [
    'idnschedule', 'idncase', 'assigned_judge', 'scheduled_datetime', 
    'judge', 'caseload', 'hearing_datetime', 'time_to_next', 'assigned', 'nat', 'lang',
    'e_28_date', 'date_of_entry', 'c_birthdate', 'gender', 'lpr', 'lawyer',
    'idnproceeding', 'osc_date', 'casetype_AOC', 'casetype_CFR',
    'casetype_REC', 'casetype_RMV', 'casetype_WHO', 'custody_D',
    'casepriority__AWC/ATD', 'casepriority__CR',
    'casepriority__RBC/D', 'casepriority__UC', 'crim__Y',
    'length_of_proceedings', 'lang_ENG', 'lang_FR', 'lang_MAN', 'lang_SP',
    'lang_TU', 'lang_other', 
    'nat_CU', 'nat_ES', 'nat_HO', 'nat_MX',
    'nat_VE', 'nat_other'
]

# Select just these columns from your DataFrame (assuming your DataFrame is named `df`)
wdf = wdf[cols_to_keep]

KeyError: "['lang_FR'] not in index"